# Slicing volume, smoothing & subsampling


Model preparation for Specfem's input using precomputed cubes. Only for rectangular area. To include rotation, check folder (Notebooks)

In [3]:
#load all necessary module
#SET PATH TO MODULE

from sys import argv
from sys import path
path.append("../..")
import numpy as np
import pandas as pd
import scipy as sp
from scipy import ndimage
from scipy import signal
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
from scipy.interpolate import RegularGridInterpolator
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from scipy.ndimage import gaussian_filter1d
import time
import copy
from tqdm import tqdm_notebook
from time import sleep
import matplotlib.pyplot as plt #add to module
import matplotlib.patches as patches#add to module


In [ ]:
#load data pregenerated data (for details see notebooks folder)
ifilename = '../ready_cubes/z10m_nam_model_vp_vs_rho_Q_props_Iban.npz'

#unpickle the data above
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

In [ ]:
#take all fields and put to gm 
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print(gm3d)

In [ ]:
#QC the volume and plot the slice 
gm3d.QCvolume(xslice=250000,yslice=580000,zslice=2000)

In [ ]:
print(gm3d)

## Slicing volume based on rectangle

In [ ]:
#create rectangle for volume slice 
#x1,x2,y1,y2
bound = (226000,250000,580000,600000)


In [ ]:
#Slice volume
#turn slicenow to false for just QC the rectangle
slgm3d = gm3d.QCslicerectangular(boundingbox=bound,zslice = 0,slicenow=True)

#delete gm3d
del gm3d

## Smoothing sliced volume

In [ ]:
#get the lenght of smoothing kernel
sigma = slgm3d.get_sigma(100)

In [ ]:
#smoothing
slgm3d.smoothXYZ(sigma[0],sigma[1],sigma[2])

In [ ]:
#QC the result
slgm3d.QCvolume(xslice=240000,yslice=590000,zslice=2000)

## Subsampling sliced volume

In [ ]:
#prepare spacing for subsampling
newdelta = slgm3d.get_deltasubsamp(100)

In [ ]:
#subsampling
#isz must half delta z 
slgm3d.subsample(isz=newdelta[2]/2,idx=newdelta[0],idy=newdelta[1],idz=newdelta[2]) 

In [ ]:
#QC the result
slgm3d.QCvolume(xslice=240000,yslice=590000,zslice=2000)

# Writing for SPECFEM MESH

In [ ]:
#write the mesh
from gnam.specutils.gmutil import gmutil

gmu = gmutil()
gmu.writeSpecfem3DMesh('specfemmesh/', slgm3d)